# Выбор локации для скважины

Цель проекта: Нужно решить, где бурить новую скважину.

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. 

***
 Задача: построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализировать возможную прибыль и риски техникой *Bootstrap.*

Условия задачи:
 - Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
 - При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
 - Бюджет на разработку скважин в регионе — 10 млрд рублей.
 - При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
 - После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

Описание данных:

`id` — уникальный идентификатор скважины;
`f0`, `f1`, `f2` — три признака точек (неважно, что они означают, но сами признаки значимы);
`product` — объём запасов в скважине (тыс. баррелей).

## Загрузка и подготовка данных

Начнем с импорта всего необходимого, функций и данных:

In [1]:
# импорт функций

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error



In [2]:
# импорт данных

df1 = pd.read_csv('geo_data_0.csv')
df2 = pd.read_csv('geo_data_1.csv')
df3 = pd.read_csv('geo_data_2.csv')

print(df1.info())
print(df2.info())
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column  

Делать предобработку данных нам не потребуется, но избавимся от стаолбца 'id' - в проекте он на не понадобится совсем. Стандартизацию численных признаков сделаем чуть позже.

In [3]:
df1 = df1.drop('id', axis=1)
df2 = df2.drop('id', axis=1)
df3 = df3.drop('id', axis=1)

Данные готовы, приступим к основной части проекта

## Обучение и проверка модели

Построим модели для каждого региона, предсказывающие объем запасов на новых скважинах.
Для нашей задачи подойдет линейная регрессия, с ней и будем работать.

 Для удобства создадим функцию построения модели:

In [4]:
# creating df names

df1.name = 'Регион 1'
df2.name = 'Регион 2'
df3.name = 'Регион 3'

In [5]:
# функция создания модели для региона

def split_and_train(df):
    results = []
    
    # ftf split
    features = df.drop('product', axis=1)
    target = df['product']
    
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)
    
    # стандартизация численных признаков
    scaler = StandardScaler()
    scaler.fit(features_train)

    features_train = scaler.transform(features_train)
    features_valid = scaler.transform(features_valid)

    # train
    model = LinearRegression()
    model.fit(features_train, target_train)
    
    predictions_valid = model.predict(features_valid)  
    
    rmse = mean_squared_error(target_valid, predictions_valid) ** 0.5
    
    mean_reserve = sum(predictions_valid) / len(predictions_valid)
           
    print(df.name)
    print('Средний запас предсказанного сырья:', mean_reserve, 'RMSE модели:', rmse)
    print('____________________________________________________________________________________')
    
    return predictions_valid, target_valid, rmse

In [6]:
predictions_valid1, target_valid1, rmse1 = split_and_train(df1)
predictions_valid2, target_valid2, rmse2 = split_and_train(df2)
predictions_valid3, target_valid3, rmse3 = split_and_train(df3)


Регион 1
Средний запас предсказанного сырья: 92.59256778438008 RMSE модели: 37.5794217150813
____________________________________________________________________________________
Регион 2
Средний запас предсказанного сырья: 68.72854689544579 RMSE модели: 0.8930992867756167
____________________________________________________________________________________
Регион 3
Средний запас предсказанного сырья: 94.96504596800506 RMSE модели: 40.02970873393434
____________________________________________________________________________________


Самая точная модель получилась у второго региона, но там же предсказан самый маленький объем запасов

## Подготовка к расчёту прибыли

In [7]:
# запишем константы

price_per_1000barrel = 450_000
development_budget = 10_000_000_000
n_points = 200

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины:

In [8]:
primary_reserve = development_budget / price_per_1000barrel / n_points
primary_reserve

111.11111111111111

Значения средних запасов, которые мы предсказали в предыдущем шаге ниже требуемого объема
***
Создадим функцию для расчета прибыли:

In [9]:
# создание функции

def evaluating_revenue(predictions, target):
    x = pd.Series(predictions, index=target.index)
    top_200 = x.sort_values(ascending=False)[:200]
    revenue = (target[top_200.index] * price_per_1000barrel).sum() - development_budget
    return revenue

# проверка
evaluating_revenue(predictions_valid1, target_valid1)
   

3320826043.1398506

## Расчёт прибыли и рисков 

Посчитаем риски и прибыль для каждого региона:

 Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли, снова, чтобы не повторять код, создадим функцию:

In [10]:
state = np.random.RandomState(12345)

In [11]:

def bootstrap_mean_rev(target, predictions, region):
    revenues= []
    
    target_boot = target.reset_index(drop=True)
    for i in range(1000):
        sub_target = target_boot.sample(n=500, replace=True, random_state=state)
        sub_predictions = predictions[sub_target.index]
        revenue = evaluating_revenue(sub_predictions, sub_target)
        revenues.append(revenue)
        
    revenues = pd.Series(revenues)
    mean_revenue = sum(revenues) / len(revenues)
    low_border = revenues.quantile(0.025)
    upper_border = revenues.quantile(0.975)
    loss_possibility = revenues[revenues<0].count() / len(revenues) * 100

    print('Регион', region)
    print('Средняя прибыль:', mean_revenue)
    print('Доверительный интервал:', low_border, '-', upper_border)
    print('Вероятность убытков:', loss_possibility)
    print("----------------------------------------------------------------")



Теперь непосредственно рассчитаем прибыль и риски убытков для каждого региона:

In [12]:
bootstrap_mean_rev(target_valid1, predictions_valid1, 1)
bootstrap_mean_rev(target_valid2, predictions_valid2, 2)
bootstrap_mean_rev(target_valid3, predictions_valid3, 3)

Регион 1
Средняя прибыль: 600735244.2611656
Доверительный интервал: 12948331.135115195 - 1231163605.791502
Вероятность убытков: 2.0
----------------------------------------------------------------
Регион 2
Средняя прибыль: 663958995.2601925
Доверительный интервал: 206476361.25177097 - 1191197684.7488472
Вероятность убытков: 0.1
----------------------------------------------------------------
Регион 3
Средняя прибыль: 597381047.9005232
Доверительный интервал: 1734929.5311780055 - 1246217960.1652355
Вероятность убытков: 2.5
----------------------------------------------------------------


Оновная задача этого проекта - предложить регион для разработки новых скважен. 

 И, кажется, что лучшим решением из трех предложенных регионов - второй: у него самая низкая вероятность рисков, да и самые высокие значения предстказаний средней прибыли.